# Part 1 (Data Collection, Processing, and Resampling)

## Imports

In [2]:
import requests
from abc import ABC, abstractmethod
import time
import binance
import pandas as pd
from requests.exceptions import ConnectTimeout
from tqdm import tqdm
import mplfinance as mpf
import os
import plotly.express as px

In [3]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

## Constants

### Time Intervals

In [4]:
from datetime import datetime

start_date = datetime(2023, 1, 1)
finish_date = datetime(2024, 1, 1)

start_seconds = int(start_date.timestamp())
finish_seconds = int(finish_date.timestamp())

start_ms = int(start_date.timestamp() * 1000)
finish_ms = int(finish_date.timestamp() * 1000)

### Folders

In [5]:
DATA_DIR = 'data/'
RAW_DATA_DIR = DATA_DIR + 'raw data/'
ROBUST_DATA_DIR = DATA_DIR + 'robust raw data/'

## Data Collection (APIs)

### Nobitex

In [ ]:
class NobitexAPI(ABC):
    base_url = 'https://chart.nobitex.ir'

    def __init__(self, data, status, errmsg=None):
        self.data = data
        self.status = status
        self.errmsg = errmsg

    def __repr__(self):
        return f"{self.__class__.__name__}(status={self.status}, data={self.data}, errmsg={self.errmsg})"

    @classmethod
    @abstractmethod
    def fetch(cls, **kwargs):
        """Fetch data from Nobitex API. Must be implemented by subclasses."""
        pass

    @classmethod
    def get(cls, endpoint, params):
        """Generic GET request method."""
        url = f"{cls.base_url}{endpoint}"
        url_with_params = requests.Request('GET', url, params=params).prepare().url
        
        # Print the full URL with parameters
        print(f"Fetching: {url_with_params}")
        
        response = requests.get(url, params=params)
        data = response.json()
        return data

class NobitexMarketHistory(NobitexAPI):
    @classmethod
    def fetch(cls, symbol, resolution, to, from_=None, countback=None, page=1):
        # Ensure resolution is in the correct format
        if isinstance(resolution, int):
            resolution = str(resolution)

        # Define maximum 40-day interval in seconds (40 days * 24 hours * 60 minutes * 60 seconds)
        max_interval = 30 * 24 * 60 * 60

        data = []
        current_from = from_
        current_to = min(to, current_from + max_interval)
        
        while current_from < to:
            endpoint = '/market/udf/history'
            params = {
                'symbol': symbol,
                'resolution': resolution,
                'from': current_from,
                'to': current_to,
                'countback': countback,
                'page': page
            }
            params = {k: v for k, v in params.items() if v is not None}
            
            # Use the generic get method from NobitexAPI
            result = cls.get(endpoint, params)
            
            # Process result
            if result.get('s') == 'ok':
                # Combine all data fields into a list of dictionaries
                combined_data = [
                    {
                        'open_time': t,
                        'open': o,
                        'high': h,
                        'low': l,
                        'close': c,
                        'volume': v,
                    }
                    for t, o, h, l, c, v in zip(result['t'], result['o'], result['h'], result['l'], result['c'], result['v'])
                ]
                data.extend(combined_data)
            elif result.get('s') == 'error':
                return cls(data=None, status='error', errmsg=result.get('errmsg'))
            elif result.get('s') == 'no_data':
                return cls(data=None, status='no_data')
            
            # Update the from and to timestamps for the next request
            current_from = current_to
            current_to = min(to, current_from + max_interval)
            time.sleep(1)  # Avoid hitting API rate limits

        return cls(data=data, status='ok')

    def to_dataframe(self):
        """Convert data to pandas DataFrame with specified columns."""
        df = pd.DataFrame(self.data)
        return df

In [ ]:
nobitex_history = NobitexMarketHistory.fetch(symbol='BTCIRT', resolution='1', from_=start_seconds, to=finish_seconds)
len(nobitex_history.data)

### Binance

#### Client

In [ ]:
binance_client = binance.Client()

#### Fetch Data

In [ ]:
def fetch_binance_data(symbol, interval, start_ms, finish_ms):
    all_klines = []

    # Approximate number of iterations (assuming 1 minute interval and 1000 klines per request)
    total_iterations = (finish_ms - start_ms) // (60000 * 1000)
    
    with tqdm(total=total_iterations) as pbar:
        while True:
            try:
                klines = binance_client.get_klines(symbol=symbol, interval=interval, startTime=start_ms, endTime=finish_ms, limit=1000)
                if not klines:
                    print("No more data returned.")
                    break

                all_klines.extend(klines)
                start_ms = klines[-1][6] + 1
                pbar.update(1)  # Update the progress bar for each iteration

                if klines[-1][6] >= finish_ms:
                    print("Reached the end of the specified time range.")
                    break
            except ConnectTimeout:
                print("Connection timed out. Retrying...")
                time.sleep(5)
            except Exception as e:
                print(f"An error occurred: {e}")
                break

    if all_klines:
        df = pd.DataFrame(all_klines, columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
        df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
        df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
        return df
    else:
        print("No data was fetched.")
        return pd.DataFrame()

In [ ]:
symbol = 'BTCUSDT'
interval = binance_client.KLINE_INTERVAL_1MINUTE

binance_df = fetch_binance_data(symbol, interval, start_ms, finish_ms)

In [ ]:
print(binance_df.shape)
binance_df.head()

In [ ]:
def load_and_preprocess_binance_data(file_name='Binance-BTCUSDT.csv'):
    global binance_df
    
    if 'binance_df' not in globals():
        file_path = os.path.join(DATA_DIR, file_name)
        
        if os.path.exists(file_path):
            binance_df = pd.read_csv(file_path)
            print(f"Loaded data from {file_path}")
        else:
            print(f"File {file_path} does not exist. Check the path.")
            return
    else:
        print("binance_df is already defined.")

    binance_df['close_time'] = pd.to_datetime(binance_df['close_time'])
    
    columns_to_convert = ['open', 'high', 'low', 'close', 'volume', 
                          'quote_asset_volume', 'number_of_trades', 
                          'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume']
    binance_df[columns_to_convert] = binance_df[columns_to_convert].apply(pd.to_numeric, errors='coerce')
    
    binance_df.dropna(inplace=True)
    print("Data preprocessing complete.")


In [ ]:
load_and_preprocess_binance_data()

#### Save CSV

In [ ]:
# start_time = pd.to_datetime('2022-11-20 15:38:00')
# end_time = pd.to_datetime('2024-03-09 13:16:00')

# binance_df = binance_df[(binance_df.index >= start_time) & (binance_df.index <= end_time)]

In [ ]:
# binance_df.to_csv(ROBUST_DATA_DIR + f'Binance-{symbol}.csv')

### Load CSV

In [ ]:
binance_df = pd.read_csv(ROBUST_DATA_DIR + f'Binance-{symbol}.csv')
binance_df['open_time'] = pd.to_datetime(binance_df['open_time'])
binance_df.set_index('open_time', inplace=True)

#### Load & Pre-Process

#### Visualization

In [ ]:
def plot_candles(df):
    if df.empty:
        print("The DataFrame is empty. No data to plot.")
        return
    
    df.index.name = 'Date'
    df = df[['open', 'high', 'low', 'close', 'volume']]
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

    mpf.plot(df, type='candle', volume=True, style='charles', title=f'Candles')

In [ ]:
plot_candles(binance_df[:100])

## Handling Missing Data

To address missing candles in a time series dataset, particularly for financial data like Bitcoin trading, it's crucial to maintain the continuity and integrity of the data series. Here are a couple of simple strategies that can be used to fill in missing data:

1. Carry Forward the Last Known Value (Last Observation Carried Forward - LOCF)
    In this method, if a time interval is missing, the data from the last available interval is carried forward to fill the gap. This approach assumes that the market conditions did not change significantly between the missing intervals.

2. Simple Average of Adjacent Data Points
    This method uses the average of the data points before and after the missing interval to fill the gap. It assumes that the missing data is likely to be similar to the average of the surrounding points.

Considerations
- Granularity: Given that the data is on a one-minute scale, gaps might not significantly impact the analysis if they are sparse.
- Market Conditions: Financial markets can change rapidly, so the chosen method should be as accurate as possible, given the limitations.

In [ ]:
def fill_missing_data(input_path, output_path, start_date, finish_date, method='interpolate'):
    df = pd.read_csv(input_path)
    
    # Rename the 'datetime' column to 'open_time'
    df.rename(columns={'datetime': 'open_time'}, inplace=True)
    
    # Set 'open_time' as the index and sort it
    df.set_index('open_time', inplace=True)
    df.sort_index(inplace=True)
    
    # Ensure the index is a DatetimeIndex
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)
    
    # Trim the dataset to the specified date range
    df = df.loc[start_date:finish_date]
    
    # Handle missing values based on the specified method
    if method == 'ffill':
        df_processed = df.resample('min').ffill()
    elif method == 'interpolate':
        df_processed = df.resample('min').interpolate(method='linear')
        df_processed.ffill(axis=0, inplace=True)  # For any remaining missing values
    else:
        raise ValueError("Invalid method. Choose either 'ffill' or 'interpolate'.")
    
    print('Converting Candles...')
    print(f'From: {df.index.min()}')
    print(f'To: {df.index.max()}')
    print(30 * '-')
    
    # Save the processed DataFrame to a new CSV file
    df_processed.to_csv(output_path)

In [ ]:
fill_missing_data(ROBUST_DATA_DIR + 'Binance-BTCUSDT.csv', ROBUST_DATA_DIR + 'Binance-BTCUSDT.csv', start_date, finish_date)
fill_missing_data(RAW_DATA_DIR + 'btctmn_m1_tabdeal.csv', ROBUST_DATA_DIR + 'Tabdeal-BTCTMN.csv', start_date, finish_date)
fill_missing_data(RAW_DATA_DIR + 'usdttmn_m1_nobitex.csv', ROBUST_DATA_DIR + 'Nobitex-USDTTMN.csv', start_date, finish_date)
fill_missing_data(RAW_DATA_DIR + 'usdttmn_m1_tabdeal.csv', ROBUST_DATA_DIR + 'Tabdeal-USDTTMN.csv', start_date, finish_date)
fill_missing_data(RAW_DATA_DIR + 'usdttmn_m1_wallex.csv', ROBUST_DATA_DIR + 'Wallex-USDTTMN.csv', start_date, finish_date)

## Data Extraction

In [48]:
btc_usdt = pd.read_csv(r'data\robust raw data\Binance-BTCUSDT.csv')
btc_tmn = pd.read_csv(r'data\robust raw data\Tabdeal-BTCTMN.csv')

### Processing OHLCV Data into a Representative Price Series

In [54]:
# Convert open_time to datetime
btc_usdt['open_time'] = pd.to_datetime(btc_usdt['open_time'])
btc_tmn['open_time'] = pd.to_datetime(btc_tmn['open_time'])

# Representative price calculations
btc_usdt['close_price'] = btc_usdt['close']
btc_tmn['close_price'] = btc_tmn['close']

btc_usdt['weighted_avg_price'] = (btc_usdt['open'] * 0.2 + btc_usdt['high'] * 0.2 + btc_usdt['low'] * 0.2 + btc_usdt['close'] * 0.4)
btc_tmn['weighted_avg_price'] = (btc_tmn['open'] * 0.2 + btc_tmn['high'] * 0.2 + btc_tmn['low'] * 0.2 + btc_tmn['close'] * 0.4)

btc_usdt['average_price'] = btc_usdt[['open', 'high', 'low', 'close']].mean(axis=1)
btc_tmn['average_price'] = btc_tmn[['open', 'high', 'low', 'close']].mean(axis=1)

# Merge the DataFrames on 'open_time'
merged_df = pd.merge(btc_usdt, btc_tmn, on='open_time', suffixes=('_usdt', '_tmn'))

### Implied USDT-TMN Extraction

In [56]:
merged_df['close_price'] = merged_df['close_price_tmn'] / merged_df['close_price_usdt']
merged_df['weighted_avg_price'] = merged_df['weighted_avg_price_tmn'] / merged_df['weighted_avg_price_usdt']
merged_df['average_price'] = merged_df['average_price_tmn'] / merged_df['average_price_usdt']

result_df = merged_df[['open_time', 'close_price', 'average_price', 'weighted_avg_price']]
result_df.set_index('open_time', inplace=True)

In [57]:
result_df

,close_price,average_price,weighted_avg_price
open_time,,,
2023-01-01 00:00:00,39492.634282,39496.233277,39495.513425
2023-01-01 00:01:00,39483.525975,39488.510432,39487.513637
2023-01-01 00:02:00,39511.618469,39497.600853,39500.404183
2023-01-01 00:03:00,39589.426957,39551.595836,39559.161716
2023-01-01 00:04:00,39585.284947,39590.330892,39589.321600
...,...,...,...
2023-12-31 23:56:00,51464.541485,51457.784534,51459.136112
2023-12-31 23:57:00,51427.345828,51439.651933,51437.189879
2023-12-31 23:58:00,51384.136896,51401.031082,51397.652103


In [58]:
result_df.to_csv(ROBUST_DATA_DIR + 'Implied-USDTTMN.csv')

## USDT-TMN Analysis

In [70]:
def convert_to_datetime(df):
    df['open_time'] = pd.to_datetime(df['open_time'], errors='coerce')
    return df

nobitex_usdttmn = convert_to_datetime(pd.read_csv(r'data/robust raw data/Nobitex-USDTTMN.csv'))
tabdeal_usdttmn = convert_to_datetime(pd.read_csv(r'data/robust raw data/Tabdeal-USDTTMN.csv'))
wallex_usdttmn = convert_to_datetime(pd.read_csv(r'data/robust raw data/Wallex-USDTTMN.csv'))
implied_usdttmn = convert_to_datetime(pd.read_csv(r'data/robust raw data/Implied-USDTTMN.csv'))

In [71]:
def process_data(df):
    df['open_time'] = pd.to_datetime(df['open_time'])
    df['close_price'] = df['close']
    df['weighted_avg_price'] = (df['open'] * 0.2 + df['high'] * 0.2 + df['low'] * 0.2 + df['close'] * 0.4)
    df['average_price'] = df[['open', 'high', 'low', 'close']].mean(axis=1)
    return df

nobitex_usdttmn = process_data(nobitex_usdttmn)
tabdeal_usdttmn = process_data(tabdeal_usdttmn)
wallex_usdttmn = process_data(wallex_usdttmn)

In [75]:
merged_data = implied_usdttmn.merge(nobitex_usdttmn[['open_time', 'close_price', 'average_price', 'weighted_avg_price']], on='open_time', how='inner', suffixes=('', '_nobitex'))
merged_data = merged_data.merge(tabdeal_usdttmn[['open_time', 'close_price', 'average_price', 'weighted_avg_price']], on='open_time', how='inner', suffixes=('', '_tabdeal'))
merged_data = merged_data.merge(wallex_usdttmn[['open_time', 'close_price', 'average_price', 'weighted_avg_price']], on='open_time', how='inner', suffixes=('', '_wallex'))

merged_data.rename(columns={
    'close_price': 'implied_close_price',
    'close_price_nobitex': 'nobitex_close_price',
    'close_price_tabdeal': 'tabdeal_close_price',
    'close_price_wallex': 'wallex_close_price',
    'average_price': 'implied_average_price',
    'average_price_nobitex': 'nobitex_average_price',
    'average_price_tabdeal': 'tabdeal_average_price',
    'average_price_wallex': 'wallex_average_price',
    'weighted_avg_price': 'implied_weighted_avg_price',
    'weighted_avg_price_nobitex': 'nobitex_weighted_avg_price',
    'weighted_avg_price_tabdeal': 'tabdeal_weighted_avg_price',
    'weighted_avg_price_wallex': 'wallex_weighted_avg_price'
}, inplace=True)

In [76]:
merged_data

,open_time,implied_close_price,implied_average_price,implied_weighted_avg_price,nobitex_close_price,nobitex_average_price,nobitex_weighted_avg_price,tabdeal_close_price,tabdeal_average_price,tabdeal_weighted_avg_price,wallex_close_price,wallex_average_price,wallex_weighted_avg_price
0,2023-01-01 00:00:00,39492.634282,39496.233277,39495.513425,39404.0,39416.00,39413.6,39572.0,39630.50,39618.8,39574.0,39541.00,39547.6
1,2023-01-01 00:01:00,39483.525975,39488.510432,39487.513637,39404.0,39415.50,39413.2,39572.0,39572.00,39572.0,39525.0,39521.25,39522.0
2,2023-01-01 00:02:00,39511.618469,39497.600853,39500.404183,39405.0,39412.75,39411.2,39572.0,39571.50,39571.6,39513.0,39519.00,39517.8
3,2023-01-01 00:03:00,39589.426957,39551.595836,39559.161716,39496.0,39461.00,39468.0,39570.0,39571.00,39570.8,39500.0,39494.25,39495.4
4,2023-01-01 00:04:00,39585.284947,39590.330892,39589.321600,39490.0,39449.25,39457.4,39570.0,39570.00,39570.0,39490.0,39491.00,39490.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
525596,2023-12-31 23:56:00,51464.541485,51457.784534,51459.136112,50998.0,50986.00,50988.4,50926.0,50929.00,50928.4,50989.0,50989.00,50989.0
525597,2023-12-31 23:57:00,51427.345828,51439.651933,51437.189879,50945.0,50951.75,50950.4,50894.0,50910.00,50906.8,50989.0,50993.75,50992.8
525598,2023-12-31 23:58:00,51384.136896,51401.031082,51397.652103,50946.0,50954.75,50953.0,50887.5,50900.25,50897.7,50989.0,50993.75,50992.8
525599,2023-12-31 23:59:00,51343.329042,51389.360170,51380.152764,51000.0,50981.75,50985.4,50881.0,50890.50,50888.6,50999.0,50994.25,50995.2


In [77]:
fig1 = px.line(
    merged_data[:2000],
    x='open_time',
    y=['implied_close_price', 'nobitex_close_price', 'tabdeal_close_price', 'wallex_close_price'],
    labels={'value': 'Close Price', 'variable': 'Source'},
    title='Comparison of Close Prices'
)
fig1.show()

In [78]:
fig2 = px.line(
    merged_data[:2000],
    x='open_time',
    y=['implied_average_price', 'nobitex_average_price', 'tabdeal_average_price', 'wallex_average_price'],
    labels={'value': 'Average Price', 'variable': 'Source'},
    title='Comparison of Average Prices'
)
fig2.show()

In [79]:
fig3 = px.line(
    merged_data[:2000],
    x='open_time',
    y=['implied_weighted_avg_price', 'nobitex_weighted_avg_price', 'tabdeal_weighted_avg_price', 'wallex_weighted_avg_price'],
    labels={'value': 'Weighted Average Price', 'variable': 'Source'},
    title='Comparison of Weighted Average Prices'
)
fig3.show()